# Data Processing

Kick out unnessesary colums like <br>
- Epidemic (This column only consists of 0 and does not add any value)
- Seasonality (This column is filled just with Winter and therefore does not add any value)

In [159]:
import numpy as np
import pandas as pd
from skrebate import ReliefF

# Read in the data
df = pd.read_csv("../../Data/Raw/inventory_management.csv")
# Make sure its correct
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Seasonality,Epidemic,Demand
0,2022-01-01,S001,P0001,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,Winter,0,115
1,2022-01-01,S001,P0002,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,Winter,0,229
2,2022-01-01,S001,P0003,Clothing,North,247,114,612,62.94,10,Snowy,1,60.08,Winter,0,157
3,2022-01-01,S001,P0004,Electronics,North,139,45,102,87.63,10,Snowy,0,85.19,Winter,0,52
4,2022-01-01,S001,P0005,Groceries,North,152,65,271,54.41,0,Snowy,0,51.63,Winter,0,59


In [160]:
# Delete both rows
df = df.drop('Seasonality',axis=1)
df = df.drop('Epidemic',axis=1)
# Show new Table
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Demand
0,2022-01-01,S001,P0001,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,115
1,2022-01-01,S001,P0002,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,229
2,2022-01-01,S001,P0003,Clothing,North,247,114,612,62.94,10,Snowy,1,60.08,157
3,2022-01-01,S001,P0004,Electronics,North,139,45,102,87.63,10,Snowy,0,85.19,52
4,2022-01-01,S001,P0005,Groceries,North,152,65,271,54.41,0,Snowy,0,51.63,59


Transform Data <br>
- Create a key for each product which consists of the Store and Product ID in Order to create a forecast for each Item -> Kick out Category and safe an extra table in order to find what Item belongs where if needed
- Categorize the dates to the week days in order to create broad forcast and not only for dates.
- Merge the Competitor Price and Price in order to reduce redundancy and improve interpretability of the end tree

In [161]:
# Create uniquie product IDs
df['PKeyID'] = df['Store ID'].astype(str) + '_' + df['Product ID'].astype(str)

In [162]:
# Check if everything went well
df.head()
# Distribution: count of each unique PKeyID
print(df['PKeyID'].value_counts())
# Number of unique PKeyIDs
print("Number of unique PKeyIDs:", df['PKeyID'].nunique())
# General statistics (for object columns, describe gives count, unique, top, freq)
print(df['PKeyID'].describe())

PKeyID
S001_P0001    18
S001_P0013    18
S001_P0002    18
S002_P0002    18
S001_P0020    18
              ..
S003_P0006    17
S003_P0005    17
S003_P0004    17
S003_P0003    17
S005_P0020    17
Name: count, Length: 100, dtype: int64
Number of unique PKeyIDs: 100
count           1722
unique           100
top       S001_P0001
freq              18
Name: PKeyID, dtype: object


In [163]:
# Drop unnessesary colum Product ID and Store ID
df = df.drop('Store ID',axis=1)
df = df.drop('Product ID',axis=1)
# Check Dataframe
df.head(2)

,Date,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Demand,PKeyID
0,2022-01-01,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,115,S001_P0001
1,2022-01-01,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,229,S001_P0002


In [164]:
# Changing the Dates to the actual Weekday since it gives us better results
df['Date'] = pd.to_datetime(df['Date'])  # Convert to datetime if not already
df['Weekday'] = df['Date'].dt.day_name()  # Create new column with weekday names
df.head()

,Date,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Demand,PKeyID,Weekday
0,2022-01-01,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,115,S001_P0001,Saturday
1,2022-01-01,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,229,S001_P0002,Saturday
2,2022-01-01,Clothing,North,247,114,612,62.94,10,Snowy,1,60.08,157,S001_P0003,Saturday
3,2022-01-01,Electronics,North,139,45,102,87.63,10,Snowy,0,85.19,52,S001_P0004,Saturday
4,2022-01-01,Groceries,North,152,65,271,54.41,0,Snowy,0,51.63,59,S001_P0005,Saturday


In [165]:
# Get rid of the unnessesary colum Date
df = df.drop('Date',axis=1)
df.head()

,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Demand,PKeyID,Weekday
0,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,115,S001_P0001,Saturday
1,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,229,S001_P0002,Saturday
2,Clothing,North,247,114,612,62.94,10,Snowy,1,60.08,157,S001_P0003,Saturday
3,Electronics,North,139,45,102,87.63,10,Snowy,0,85.19,52,S001_P0004,Saturday
4,Groceries,North,152,65,271,54.41,0,Snowy,0,51.63,59,S001_P0005,Saturday


In [166]:
# Save Category mapping to PKeyID
# Drop duplicates only save one per PKeyID
#mapping = df[['PKeyID', 'Category']].drop_duplicates()

# Save to a text file (CSV format)
#mapping.to_csv('../../Data/Transformed/Julius/PKeyID_Category_Mapping.txt', index=False, sep='\t')

In [167]:
# Drop unnessesary data colum Category
#df = df.drop('Category',axis=1)
#df.head()

In [168]:
# Since Price and Competitor Price are highly correlated we are buiulind the diffrence for better interpretability
df['Price_Diff'] = (df['Price'] - df['Competitor Pricing']).round(3)
df.head()

,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Demand,PKeyID,Weekday,Price_Diff
0,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,115,S001_P0001,Saturday,-13.01
1,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,229,S001_P0002,Saturday,-11.86
2,Clothing,North,247,114,612,62.94,10,Snowy,1,60.08,157,S001_P0003,Saturday,2.86
3,Electronics,North,139,45,102,87.63,10,Snowy,0,85.19,52,S001_P0004,Saturday,2.44
4,Groceries,North,152,65,271,54.41,0,Snowy,0,51.63,59,S001_P0005,Saturday,2.78


In [169]:
# Drop Price and Competitor Price? Maybe

Encoding the Data <br>
- Encoding the Data especially the Date, Weather in order to transform the data to a computer readable format

In [170]:
from sklearn.preprocessing import LabelEncoder
# Now we encode everything so that ML algorithms can perform on the data
label_encoders = {}

# Columns you want to encode
cols_to_encode = ["Category", "Region", "Weather Condition", "Weekday", "PKeyID"]

# Encode each column and store the encoder
for col in cols_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save the encoder

# Now your DataFrame is encoded and still available
print(df.head())

   Category  Region  Inventory Level  Units Sold  Units Ordered  Price  \
0         1       1              195         102            252  72.72   
1         0       1              117         117            249  80.16   
2         0       1              247         114            612  62.94   
3         1       1              139          45            102  87.63   
4         3       1              152          65            271  54.41   

   Discount  Weather Condition  Promotion  Competitor Pricing  Demand  PKeyID  \
0         5                  1          0               85.73     115       0   
1        15                  1          1               92.02     229       1   
2        10                  1          1               60.08     157       2   
3        10                  1          0               85.19      52       3   
4         0                  1          0               51.63      59       4   

   Weekday  Price_Diff  
0        2      -13.01  
1        2      -1

In [171]:
for col, le in label_encoders.items():
    mapping = pd.DataFrame({
        'Original': le.classes_,
        'Encoded': le.transform(le.classes_)
    })
    print(f"Mapping for {col}:")
    display(mapping)

Mapping for Category:


,Original,Encoded
0,Clothing,0
1,Electronics,1
2,Furniture,2
3,Groceries,3
4,Toys,4


Mapping for Region:


,Original,Encoded
0,East,0
1,North,1
2,South,2
3,West,3


Mapping for Weather Condition:


,Original,Encoded
0,Cloudy,0
1,Snowy,1
2,Sunny,2


Mapping for Weekday:


,Original,Encoded
0,Friday,0
1,Monday,1
2,Saturday,2
3,Sunday,3
4,Thursday,4
5,Tuesday,5
6,Wednesday,6


Mapping for PKeyID:


,Original,Encoded
0,S001_P0001,0
1,S001_P0002,1
2,S001_P0003,2
3,S001_P0004,3
4,S001_P0005,4
...,...,...
95,S005_P0016,95
96,S005_P0017,96
97,S005_P0018,97
98,S005_P0019,98


In [172]:
# Save the label_encoders mappings as a text file
with open('../../Data/Transformed/Julius/label_encodings.txt', 'w') as f:
    for col, encoder in label_encoders.items():
        f.write(f"{col}:\n")
        for idx, label in enumerate(encoder.classes_):
            f.write(f"  {label}: {idx}\n")
        f.write("\n")

In [173]:
df.head()

,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Demand,PKeyID,Weekday,Price_Diff
0,1,1,195,102,252,72.72,5,1,0,85.73,115,0,2,-13.01
1,0,1,117,117,249,80.16,15,1,1,92.02,229,1,2,-11.86
2,0,1,247,114,612,62.94,10,1,1,60.08,157,2,2,2.86
3,1,1,139,45,102,87.63,10,1,0,85.19,52,3,2,2.44
4,3,1,152,65,271,54.41,0,1,0,51.63,59,4,2,2.78


In [174]:
# Use the ReliefF Algorithm to get insights in the feature importance, this can only be applied after all values are encoded and fits well
# in this scenario due to our binary classification problem
# Separate target variable and features
X = df.drop(['Promotion'], axis=1)
y = df['Promotion']

# Apply the algorithm
relieff = ReliefF(n_neighbors=100, n_features_to_select=X.shape[1])
relieff.fit(X.values, y.values)

# Output the feature importance scores
scores = pd.Series(relieff.feature_importances_, index=X.columns)
print("Feature Importance Scores (sorted):")
print(scores.sort_values(ascending=False))

Feature Importance Scores (sorted):
Discount              0.435017
Demand                0.070413
Units Sold            0.042595
Units Ordered         0.030735
Weekday               0.021638
Weather Condition     0.015377
Inventory Level       0.000849
Price_Diff           -0.004585
Competitor Pricing   -0.005241
Price                -0.008862
Category             -0.010761
PKeyID               -0.011309
Region               -0.020180
dtype: float64


In [175]:
df.to_csv('../../Data/Transformed/Julius/inventory_management_transformed.csv', index=False)